mltable because: "A fast and efficient engine to materialize the data into a Pandas or Spark dataframe," https://learn.microsoft.com/en-us/azure/machine-learning/how-to-mltable?tabs=cli%2Cpandas%2Cadls

https://learn.microsoft.com/en-us/azure/machine-learning/v1/how-to-data-ingest-adf
"The Data Factory pipeline saves the prepared data to your cloud storage (such as Azure Blob or Azure Data Lake).
Consume your prepared data in Azure Machine Learning by,

    Invoking an Azure Machine Learning pipeline from your Data Factory pipeline.
    OR
    Creating an Azure Machine Learning datastore.- In this Turtorial You will ..
- Prerequisites:
-
- (re)train model (y)
- make new model (with web service input and output modules?)
- where to get experiment/model info (if neccesary):
    - 	 "Click on Use Web Service to go to the Consume tab in the Menu."
    - "Copy the values of the Primary Key and the Request-Response fields and save them in a document.""
    -
 https://learn.microsoft.com/en-us/azure/machine-learning/v1/how-to-set-up-training-targets

"Trainieren und Bereitstellen eines Machine Learning-Modells in Azure Machine Learning. Verwenden Sie eines dieser drei Azure Machine Learning-Tutorials:

    Option A: Code
    Option B: Designer
    Option C: Automatisiertes ML

" from here https://learn.microsoft.com/de-de/power-bi/connect-data/service-aml-integrate